## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import requests
import os
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
# Use different models, falling back to OpenAI if others don't work
sales_agent1 = Agent(name="OpenAI Sales Agent", instructions=instructions1, model="gpt-4o-mini")
sales_agent2 = Agent(name="OpenAI Sales Agent 2", instructions=instructions2, model="gpt-4o")  
sales_agent3 = Agent(name="OpenAI Sales Agent 3", instructions=instructions3, model="gpt-3.5-turbo")

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [ ]:
@function_tool
def send_simple_notification(message: str) -> Dict[str, str]:
    """ Send a simple notification with the given message to all sales prospects """
    url = "https://api.pushover.net/1/messages.json"
    
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": message,
        "title": "Sales Email"
    }
    
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return {"status": "success", "message": "Notification sent successfully"}
    else:
        return {"status": "error", "message": f"Error {response.status_code}: {response.text}"}

In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
notification_tools = [send_simple_notification]

In [ ]:
instructions = "You are a notification sender. You receive the body of an email to be sent. \
You use the send_simple_notification tool to send the email content as a clean, readable notification. \
Do not add any formatting or subjects - just send the email content directly."


notification_agent = Agent(
    name="Notification Sender",
    instructions=instructions,
    tools=[send_simple_notification],
    model="gpt-4o-mini",
    handoff_description="Send the email content as a simple notification")

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [notification_agent]

In [ ]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.

CRITICAL: You must complete ALL steps before making ANY handoff. You can only make ONE handoff total.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review ALL three drafts and choose the single best email using your judgment of which one is most effective. DO NOT hand off during this step.

3. Single Handoff: Hand off ONLY the winning email draft text to the 'Notification Sender' agent. 

FORBIDDEN ACTIONS:
- DO NOT hand off each email as you generate it
- DO NOT make multiple handoffs
- DO NOT hand off all three emails
- You can make ONLY ONE handoff with ONLY the best email text

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Notification Sender — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[notification_agent],
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message, max_turns=20)

## Check out the trace:

https://platform.openai.com/traces

In [ ]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [ ]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Careful Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[notification_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

# Test Guardrail Behavior - Choose one message:

# Option 1: Message WITHOUT personal name - Guardrail will allow execution
# message = "Send out a cold sales email addressed to Dear CEO from Sales Team"

# Option 2: Message WITH personal name - Guardrail will BLOCK with InputGuardrailTripwireTriggered
message = "Send out a cold sales email addressed to Dear CEO from Alice"

print(f"Testing with message: {message}")
if "Alice" in message or "from " in message.split("from ")[-1] and len(message.split("from ")[-1].split()) == 1:
    print("Expected: Guardrail will BLOCK execution (personal name detected)")
else:
    print("Expected: Guardrail allows execution (no personal name detected)")

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message, max_turns=20)

## Check out the trace:

https://platform.openai.com/traces

In [ ]:

# This message should pass the guardrail (no personal name)
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

print(f"Testing with message: {message}")
print("Expected: Guardrail allows execution (no personal name detected)")

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message, max_turns=20)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>